## Testing that polymerist is importable

In [1]:
import logging
logging.basicConfig(
    level=logging.INFO
)

import polymerist as ps
from polymerist.genutils.importutils import module_hierarchy

# print(module_hierarchy(ps))

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2023.09.5 jupyter extensions


[11:37:49] WARNING: not removing hydrogen atom with dummy atom neighbors
INFO:polymerist.smileslib.functgroups:Loading functional group SMARTS data from LUT


In [2]:
import MDAnalysis as mda

import warnings
# suppress some MDAnalysis warnings when writing PDB files
warnings.filterwarnings('ignore')

## Parameterizing PDB system

### Initializing paths and working directory

In [3]:
from pathlib import Path
from polymerist.genutils.fileutils.pathutils import assemble_path


working_dir = Path.cwd() # can change this to wherever your files are
mol_name = 'SPW_1m_JC'

pdb_path = assemble_path(working_dir, mol_name, extension='pdb') 
sdf_path = assemble_path(working_dir, mol_name, extension='sdf') 
inc_path = assemble_path(working_dir, mol_name, extension='pkl') 

### Load Topology from PDB ( no overhead since not using long molecules), check that molecules exist

In [4]:
from openff.toolkit import Molecule, Topology, ForceField

from polymerist.mdtools.openfftools import topology


if sdf_path.exists():
    offtop = topology.topology_from_sdf(sdf_path)
else:
    assert(pdb_path.exists())
    offtop = Topology.from_pdb(pdb_path)

# mol = next(offtop.molecules)
# mol.visualize(backend='nglview')

INFO:polymerist.mdtools.openfftools.topology:Loading serialized SDF Topology from /home/bamo6610/Documents/openff-dev/SPW_openMM/SPW_1m_JC.sdf


### Parameterize system w/ OpenFF

In [5]:
import pickle
from openff.interchange import Interchange

from polymerist.unitutils.interop import openff_to_openmm
from polymerist.mdtools.openfftools import boxvectors
from polymerist.mdtools.openfftools import TKREGS, FF_PATH_REGISTRY


# load interchange from file if already extant, otherwise make a new one and save it
if inc_path.exists():
    with inc_path.open('rb') as file:
        inc  = pickle.load(file)
else:
    ff = ForceField('openff-2.0.0.offxml', 'tip3p.offxml') # load generic Sage + TIP3P force fields
    inc = ff.create_interchange(topology=offtop, toolkit_registry=TKREGS['OpenEye Toolkit']) # convert to interchange for export
    inc.box = boxvectors.get_topology_bbox(offtop)

    if not sdf_path.exists():
        topology.topology_to_sdf(sdf_path, inc.topology)

    with inc_path.open('wb') as file:
        pickle.dump(inc, file)

# extract OpenMM-specific objects to initialize simulations later
ommtop = inc.to_openmm_topology()
ommsys = inc.to_openmm_system(combine_nonbonded_forces=False, add_constrained_forces=True)
ommpos = openff_to_openmm(inc.positions)

### Create ion subsystem

In [6]:
# from openmm.openmm import System
# import parmed as pmd

# pmd_system=pmd.openmm.load_topology(ommtop, ommsys)

# u=mda.Universe(pmd_system)
# ions=u.select_atoms('resname NA CL')
# ions=ions.convert_to('PARMED')


In [7]:
# import openmm.app as app
# ionsys=ions.createSystem(topology=ommtop,nonbondedMethod=app.NoCutoff, constraints=None)

In [8]:
from openmm.app.topology import Topology

ionindex=[]

for atom in ommtop.atoms():
    res=atom.residue.name
    if res == 'NA' or res == 'CL':
        indx=atom.index
        ionindex.append(indx)

In [10]:
len(ionindex)

130

## Applying constraints - Flat-bottom Potentials

In [10]:
from openmm.unit import kilojoule_per_mole, nanometer
import openmm
# k=4184*(kilojoule_per_mole/nanometer**2)
# r=2.4*nanometer
fb_force = openmm.CustomExternalForce('0.5*4184*max(0, r-2.4)^2; r=abs(z-7.2)')
# fb_force.addPerParticleParameter("zi")

In [11]:
ommsys.addForce(fb_force)
for i in ionindex:
    fb_force.addParticle(i,[])

In [12]:
# ommsys.getNumParticles()

In [13]:
# ionsys.addForce(fb_force)

In [14]:
# for i in range(ionsys.getNumParticles()):
#     fb_force.addParticle(i, [])

In [15]:
# ionsys.getNumParticles()

## Creating OpenMM simulations

### Defining simulation parameters

In [16]:
from polymerist.mdtools.openmmtools.parameters import SimulationParameters, ThermoParameters, IntegratorParameters, ReporterParameters
from polymerist.mdtools.openmmtools.thermo import EnsembleFactory

from openmm.unit import kelvin, atmosphere, nanosecond, picosecond, femtoseconds


all_omm_sims : dict[str, SimulationParameters] = {
    'equil_sim' : SimulationParameters(
        integ_params=IntegratorParameters(
            time_step=2*femtoseconds,
            total_time=10*picosecond,
            num_samples=50,
        ),
        thermo_params=ThermoParameters(
            ensemble='NVT',
            temperature=300 * kelvin,
        ),
        reporter_params=ReporterParameters(),
    ),
    'prod_sim' : SimulationParameters(
        integ_params=IntegratorParameters(
            time_step=2*femtoseconds,
            total_time=100*picosecond,
            num_samples=50,
        ),
        thermo_params=ThermoParameters(
            ensemble='NVT',
            temperature=300 * kelvin,
        ),
        reporter_params=ReporterParameters(),
    ),
}    

# for path_name, sim_params in all_omm_sims.items():
    # sim_params.to_file(assemble_path(working_dir, path_name, postfix='params', extension='json'))

### Run the defined simulation in series

In [17]:
from polymerist.mdtools.openmmtools.execution import run_simulation_schedule


omm_sim_dir = Path('OpenMM_sims_FBPs')
omm_sim_dir.mkdir(exist_ok=True)

history = run_simulation_schedule(
    working_dir=omm_sim_dir,
    schedule=all_omm_sims,
    init_top=ommtop,
    init_sys=ommsys,
    init_pos=ommpos,
    return_history=True
)

INFO:polymerist.mdtools.openmmtools.execution:Initializing simulation 1/2 ("equil_sim")
INFO:polymerist.mdtools.openmmtools.thermo:Created LangevinMiddleIntegrator for NVT (Canonical) ensemble
INFO:polymerist.mdtools.openmmtools.preparation:Setting simulation state
INFO:polymerist.mdtools.openmmtools.reporters:Prepared DCDReporter which reports to OpenMM_sims_FBPs/equil_sim/equil_sim_trajectory.dcd
INFO:polymerist.mdtools.openmmtools.reporters:Prepared CheckpointReporter which reports to OpenMM_sims_FBPs/equil_sim/equil_sim_checkpoint.chk
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateReporter which reports to OpenMM_sims_FBPs/equil_sim/equil_sim_state.xml
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateDataReporter which reports to OpenMM_sims_FBPs/equil_sim/equil_sim_state_data.csv
INFO:polymerist.mdtools.openmmtools.preparation:Setting positions in Context
INFO:polymerist.mdtools.openmmtools.execution:Performing energy minimization (initial PE = 2.3407988346